In [1]:
import gdown
import os
import zipfile

import rasterio
import numpy as np
from tqdm import tqdm
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from datetime import datetime
import pandas as pd

# Country Data

In [2]:
countries_zip_url = "https://drive.google.com/uc?id=1UQzdO7suT0BnwKBeNybMG97vM9GIDogA"
countries_zip_file_path = "../../allCountries.zip"

# Download the ZIP file if it doesn't exist; otherwise, proceed to read the TXT file.
if not os.path.exists(countries_zip_file_path):
    gdown.download(countries_zip_url, countries_zip_file_path, quiet=False)

with zipfile.ZipFile(countries_zip_file_path) as z:
    countries_txt_filename = "allCountries.txt"

    with z.open(countries_txt_filename) as txt_file:
        countries_df = pd.read_csv(txt_file, sep="\t", header=None)


# https://download.geonames.org/export/dump/
countries_df.columns = [
    "geonameid",
    "name",
    "asciiname",
    "alternatenames",
    "latitude",
    "longitude",
    "feature class",
    "feature code",
    "iso alpha 2",
    "cc2",
    "admin1 code",
    "admin2 code",
    "admin3 code",
    "admin4 code",
    "population",
    "elevation",
    "dem",
    "timezone",
    "modification date",
]

print(f"\nshape: {countries_df.shape}")
countries_df.head()

Downloading...
From (original): https://drive.google.com/uc?id=1UQzdO7suT0BnwKBeNybMG97vM9GIDogA
From (redirected): https://drive.google.com/uc?id=1UQzdO7suT0BnwKBeNybMG97vM9GIDogA&confirm=t&uuid=e079b820-6c48-4b1e-b8c5-c3419dbf7680
To: /Users/jiechen/Projects/Py_Code/allCountries.zip
100%|██████████| 402M/402M [00:14<00:00, 27.3MB/s] 
/var/folders/xx/1kmddksd2yzbvvwcjw3jbbbr0000gn/T/ipykernel_9575/1495126749.py:12: DtypeWarning: Columns (9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  countries_df = pd.read_csv(txt_file, sep="\t", header=None)



shape: (12950185, 19)


,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,iso alpha 2,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
0,2994701,Roc Meler,Roc Meler,"Roc Mele,Roc Meler,Roc Mélé",42.58765,1.74180,T,PK,AD,"AD,FR",02,NaN,NaN,NaN,0,2811.0,2348,Europe/Andorra,2023-10-03
1,3017832,Pic de les Abelletes,Pic de les Abelletes,"Pic de la Font-Negre,Pic de la Font-Nègre,Pic ...",42.52535,1.73343,T,PK,AD,FR,A9,66,663.0,66146.0,0,NaN,2411,Europe/Andorra,2014-11-05
2,3017833,Estany de les Abelletes,Estany de les Abelletes,"Estany de les Abelletes,Etang de Font-Negre,Ét...",42.52915,1.73362,H,LK,AD,FR,A9,NaN,NaN,NaN,0,NaN,2260,Europe/Andorra,2014-11-05
3,3023203,Port Vieux de la Coume d’Ose,Port Vieux de la Coume d'Ose,"Port Vieux de Coume d'Ose,Port Vieux de Coume ...",42.62568,1.61823,T,PASS,AD,NaN,00,NaN,NaN,NaN,0,NaN,2687,Europe/Andorra,2014-11-05
4,3029315,Port de la Cabanette,Port de la Cabanette,"Port de la Cabanette,Porteille de la Cabanette",42.60000,1.73333,T,PASS,AD,"AD,FR",B3,09,91.0,9139.0,0,NaN,2379,Europe/Andorra,2014-11-05


## EUI

In [3]:
eui_url = "https://drive.google.com/uc?id=12qGq_DLefI1RihIF_RKQUyJtm480-xRC"
eui_df = pd.read_csv(eui_url)

print(f"shape: {eui_df.shape}")
eui_df.head()

shape: (482, 5)


,City,Geonames ID,Country,Residential EUI (kWh/m2/year),Non-residential EUI (kWh/m2/year)
0,Nha Trang,1572151,Vietnam,59.096065,112.778867
1,Aberdeen,2657832,United Kingdom,231.302877,259.832393
2,Abidjan,2293538,Cote d'Ivoire,73.830819,105.622137
3,Abu Dhabi,292968,United Arab Emirates,128.447899,226.725457
4,Abuja,2352778,Nigeria,63.955819,103.009079


In [4]:
df = pd.merge(
    countries_df, eui_df, left_on="geonameid", right_on="Geonames ID", how="inner"
)

In [5]:
df = df[["geonameid", "latitude", "longitude"]]

In [6]:
df

,geonameid,latitude,longitude
0,292968,24.45118,54.39696
1,1138958,34.52813,69.17233
2,3183875,41.32750,19.81889
3,616052,40.18111,44.51361
4,2240449,-8.83682,13.23432
...,...,...,...
477,1018725,-29.12107,26.21400
478,3369157,-33.92584,18.42322
479,909137,-15.40669,28.28713
480,890299,-17.82772,31.05337


In [8]:
# df.to_csv("oringinal_datapoints.csv", index=False)

## New Population Code

In [100]:
import geopandas as gpd
from shapely.geometry import Point
import rasterio
from rasterstats import zonal_stats
from shapely.geometry import box

# Step 1: Load the data
# 1.1 Read the dataset containing 482 points
points_gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(
        df.longitude, df.latitude
    ),  # Create geometric points from longitude and latitude
    crs="EPSG:4326",  # Set the coordinate reference system to WGS84
)

# Step 2: Create buffer zones
# Create a rectangle buffer of 2km radius around each point


def create_rectangle(lat, lon, radius_km):
    lat_offset = radius_km / 111.32
    lon_offset = radius_km / (111.32 * np.cos(np.radians(lat)))
    lat_min, lat_max = lat - lat_offset, lat + lat_offset
    lon_min, lon_max = lon - lon_offset, lon + lon_offset
    return box(lon_min, lat_min, lon_max, lat_max)


points_gdf["geometry"] = points_gdf.apply(
    lambda row: create_rectangle(row["latitude"], row["longitude"], radius_km=2), axis=1
)

# Step 3: Calculate population density
# Load the global population density raster data
raster_path = "GHS_POP_E2020_GLOBE_R2023A_4326_30ss_V1_0.tif"

# Align the projection of points with the raster file
with rasterio.open(raster_path) as src:
    points_gdf = points_gdf.to_crs(
        src.crs
    )  # Convert point geometries to match raster projection

# Use rasterstats to calculate the average population density for each buffer
stats = zonal_stats(
    points_gdf,  # Buffer geometries
    raster_path,  # Raster file path
    stats=["mean"],  # Calculate the mean value
    all_touched=True,  # Include partially covered pixels
    geojson_out=False,  # Do not return results as GeoJSON
)

# Add the calculated population density to the GeoDataFrame
points_gdf["population_density"] = [
    stat["mean"] if stat["mean"] is not None else 0 for stat in stats
]

pop_df = points_gdf[["geonameid", "latitude", "longitude", "population_density"]]

In [101]:
pop_df

,geonameid,latitude,longitude,population_density
0,292968,24.45118,54.39696,3984.920922
1,1138958,34.52813,69.17233,22388.694696
2,3183875,41.32750,19.81889,10903.651198
3,616052,40.18111,44.51361,5175.170524
4,2240449,-8.83682,13.23432,24082.957531
...,...,...,...,...
477,1018725,-29.12107,26.21400,1987.197188
478,3369157,-33.92584,18.42322,1712.930254
479,909137,-15.40669,28.28713,7579.676857
480,890299,-17.82772,31.05337,4699.717804


### Validation

In [102]:
import geopandas as gpd
from shapely.geometry import box
from rasterio.features import geometry_mask
import rasterio
from rasterstats import zonal_stats

# Verification point
latitude = -11.66089
longitude = 27.47938
radius_km = 2  # Buffer radius

# Load the raster file
raster_path = "GHS_POP_E2020_GLOBE_R2023A_4326_30ss_V1_0.tif"
with rasterio.open(raster_path) as src:
    affine = src.transform

    # Create a rectangular buffer around the point
    lat_offset = radius_km / 111.32
    lon_offset = radius_km / (111.32 * np.cos(np.radians(latitude)))
    lat_min = latitude - lat_offset
    lat_max = latitude + lat_offset
    lon_min = longitude - lon_offset
    lon_max = longitude + lon_offset

    # Create the rectangular buffer polygon
    buffer_polygon = box(lon_min, lat_min, lon_max, lat_max)
    buffer_gdf = gpd.GeoDataFrame({"geometry": [buffer_polygon]}, crs="EPSG:4326")
    buffer_gdf = buffer_gdf.to_crs(src.crs)  # Reproject to match the raster CRS

    # Create a mask for the buffer zone in the raster
    mask_data = geometry_mask(
        [buffer_polygon], transform=src.transform, invert=True, out_shape=src.shape
    )

    # Extract pixel values within the buffer zone using the mask
    buffer_pixels = src.read(1)[mask_data]

    # Filter out invalid pixel values
    valid_pixels = (
        buffer_pixels[buffer_pixels != src.nodata]
        if src.nodata is not None
        else buffer_pixels[~np.isnan(buffer_pixels)]
    )

    # Calculate the weighted mean of valid pixel values
    if valid_pixels.size > 0:
        population_density_validation_weighted = valid_pixels.mean()
    else:
        population_density_validation_weighted = 0

# Output validation results (based on the improved geometry_mask logic)
print(f"Verification Point: Latitude = {latitude}, Longitude = {longitude}")
print(f"Weighted Mean (Validation Code): {population_density_validation_weighted}")

# Calculate the mean using zonal_stats
stats = zonal_stats(
    [buffer_polygon],  # Single buffer polygon
    raster_path,
    stats=["mean", "sum", "count"],  # Calculate multiple statistics
    all_touched=True,  # Include partially covered pixels
)
zonal_stats_mean = stats[0]["mean"]
print(f"Mean Calculated by zonal_stats: {zonal_stats_mean}")

# Compare the results of the validation code and zonal_stats
print(
    f"Difference between Validation Code and zonal_stats: {abs(population_density_validation_weighted - zonal_stats_mean)}"
)

Verification Point: Latitude = -11.66089, Longitude = 27.47938
Weighted Mean (Validation Code): 9074.529462654542
Mean Calculated by zonal_stats: 8977.453020250985
Difference between Validation Code and zonal_stats: 97.07644240355694
